<a href="https://colab.research.google.com/github/vishnubanna/stocktrader/blob/master/stock_analysis_certainty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import requests
from math import *

import time
import datetime as dt
import numpy as np
import multiprocessing as mp

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
#from mpl_finance import candlestick_ohlc
%matplotlib inline

from sklearn import preprocessing, svm
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.linear_model import LinearRegression

import pandas as pd
from pandas_datareader import data as pdr
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import io

import fix_yahoo_finance as yf

In [0]:
def MACD_strat(df):
    macda = np.ndarray.flatten(np.array(df.MACD))
    chakina = np.ndarray.flatten(np.array(df.cmoveA3))
    macdtesta = np.ndarray.flatten(np.array(df.MACDT))
    closea = np.ndarray.flatten(np.array(df.Close))
    datea = np.ndarray.flatten(np.array(df.Date))
    
    #funds = int(closea[0]) + 50
    funds = int(closea[0]) + 500
    
    money = funds
    stocks = 0
    bought = 0
    sells = 0 
    holds = 0 
    uncertain = 0
    buys = 0
    count = 0 
    start = funds
    buy_price = funds
    
    length = len(closea)
    
    print(funds)
    
    for i in range(length):
        macd = macda[i]
        chakin = chakina[i]
        macdt = macdtesta[i]
        price = closea[i]
        date = datea[i]
        count = count + 1
          
        if macd >= 0 and stocks == 0:
            if chakin > 0 and macdt >= 0 and money >= price:
                print('buyat: money {}, condition {}, price {} , date {}, count {}\n\n'.format(money, macd, price, date, count))
                money, stocks, buy_price = buy(money, price, stocks, buy_price)
                buys = buys + 1
            else:
                holds = holds + 1
        elif macd < 0 and stocks == 1:
            if chakin < 0 and macdt < 0:
                print('sellat: money {}, condition {}, price {} , count {}\n\n'.format(money, macd, price, date))
                money, stocks, sells = sell(money, price, stocks, sells, buy_price)
                sells = sells + 1
            else:
                holds = holds + 1
        else: 
            holds = holds + 1
        
        if count == (length-1) and stocks == 1:
            money, stocks, sells = sell(money, price, stocks, sells, buy_price)
            print('--> forced sellat: money {}, condition {}, price {} , count {}\n\n'.format(money, macd, price, date)) 
    
    print(length, money, stocks, sells, uncertain , holds, buys)
    print(count)
    profit = (money - start)/start * 100
    print('profit', profit) 
    count = 0

    return


In [0]:
def MACD_basic(df):
    # this would require extra money to be in the account, and is risky, but done right it has alot of potential. 
    macda = np.ndarray.flatten(np.array(df.MACD))
    chakina = np.ndarray.flatten(np.array(df.cmoveA))
    macdtesta = np.ndarray.flatten(np.array(df.MACDT))
    closea = np.ndarray.flatten(np.array(df.Close))
    datea = np.ndarray.flatten(np.array(df.Date))
    
    #funds = int(closea[0]) + 50
    funds = int(closea[0]) + 500
    
    money = funds
    stocks = 0
    bought = 0
    sells = 0 
    holds = 0 
    uncertain = 0
    buys = 0
    count = 0 
    start = funds
    buy_price = funds
    
    length = len(closea)
    for i in range(length):
        macd = macda[i]
        chakin = chakina[i]
        macdt = macdtesta[i]
        price = closea[i]
        date = datea[i]
        count = count + 1
          
        if macd >= 0 and money >= price and stocks == 0:
            #if chakin > 0 and macdt >= 0:
            print('buyat: money {}, condition {}, price {} , count {}\n\n'.format(money, macd, price, date))
            money, stocks, buy_price = buy(money, price, stocks, buy_price)
            buys = buys + 1
            #else:
                #holds = holds + 1
        elif macd < 0 and stocks == 1:
            #if chakin < 0 and macdt < 0:
            print('sellat: money {}, condition {}, price {} , count {}\n\n'.format(money, macd, price, date))
            money, stocks, sells = sell(money, price, stocks, sells, buy_price)
            sells = sells + 1
            #else:
                #holds = holds + 1
        else: 
            holds = holds + 1
        
        if count == (length-1) and stocks == 1:
            money, stocks, sells = sell(money, price, stocks, sells, buy_price)
            print('--> forced sellat: money {}, condition {}, price {} , count {}\n\n'.format(money, macd, price, date))
        
    print(length, money, stocks, sells, uncertain , holds, buys)
    print(count)

    profit = (money - start)/start * 100
    print('profit', profit) 
    count = 0

In [0]:
def certainty_basic(df): #good for stable companies 
    #####++>>>>>>>>>>>>>> make a buy certainty and a sell certianty
    # th
    macda = np.ndarray.flatten(np.array(df.MACD))
    #chakina = np.ndarray.flatten(np.array(df.cmoveA))
    #macdtesta = np.ndarray.flatten(np.array(df.MACDT))
    certa = np.ndarray.flatten(np.array(df.certainty))
    closea = np.ndarray.flatten(np.array(df.Close))
    datea = np.ndarray.flatten(np.array(df.Date))
    
    #funds = int(closea[0]) + 50
    funds = int(closea[0]) + 500
    
    money = funds
    stocks = 0
    bought = 0
    sells = 0 
    holds = 0 
    uncertain = 0
    uncertsells = 0
    buys = 0
    count = 0 
    start = funds
    buy_price = funds
    
    print(funds)
    
    length = len(closea)
    for i in range(length):
        macd = macda[i]
        cert = certa[i]
        price = closea[i]
        date = datea[i]
        count = count + 1
          
        if macd > 0 and stocks == 0 and money >= price:
        #if cert < 4 and macd == 4 and stocks == 0 and money >= price:    
            print('buyat: money {}, condition {}, price {} , date {}, count {}\n\n'.format(money, cert, price, date, count))
            money, stocks, buy_price = buy(money, price, stocks, buy_price)
            buys = buys + 1
        elif cert >= 8 and stocks == 1:
#             if (price >= buy_price):
#                 print('certain sellat: money {}, condition {}, price {} , count {}\n\n'.format(money, cert, price, date))
#                 money, stocks, sells = sell(money, price, stocks, sells, buy_price)
#                 sells = sells + 1
            holds = holds + 1
        elif cert <= 3 and stocks == 1:
            if (price > buy_price):
                print('certain sellat: money {}, condition {}, price {} , count {}\n\n'.format(money, cert, price, date))
                money, stocks, sells = sell(money, price, stocks, sells, buy_price)
                sells = sells + 1
#         elif cert <= 6 and stocks == 1:
#             if (price < buy_price):
#                 print('uncertain sellat: money {}, condition {}, price {} , count {}\n\n'.format(money, cert, price, date))
#                 money, stocks, sells = sell(money, price, stocks, sells, buy_price)
#                 uncertsells = uncertsells + 1
        else: 
            holds = holds + 1
        
        if count == (length) and stocks == 1:
            money, stocks, sells = sell(money, price, stocks, sells, buy_price)
            print('--> forced sellat: money {}, condition {}, price {} , count {}\n\n'.format(money, cert, price, date)) 
    
    print(length, money, stocks, sells, uncertain, uncertsells , holds, buys)
    print(count)
    profit = (money - start)/start * 100
    print('profit', profit) 
    count = 0

    return
def certainty_complex(df): #fix add delta certainty
    #####++>>>>>>>>>>>>>> make a buy certainty and a sell certianty
    # th
    macda = np.ndarray.flatten(np.array(df.MACD_certinty))
    #chakina = np.ndarray.flatten(np.array(df.cmoveA))
    #macdtesta = np.ndarray.flatten(np.array(df.MACDT))
    certa = np.ndarray.flatten(np.array(df.certainty))
    closea = np.ndarray.flatten(np.array(df.Close))
    datea = np.ndarray.flatten(np.array(df.Date))
    
    #funds = int(closea[0]) + 50
    funds = int(closea[0]) + 500
    
    money = funds
    stocks = 0
    bought = 0
    sells = 0 
    holds = 0 
    uncertain = 0
    uncertsells = 0
    buys = 0
    count = 0 
    start = funds
    buy_price = funds
    
    print(funds)
    
    length = len(closea)
    for i in range(length):
        macd = macda[i]
        cert = certa[i]
        price = closea[i]
        date = datea[i]
        count = count + 1
          
        #if macd < 2 and stocks == 0 and money >= price:
        if (macd < 2 and cert < 1.3) and stocks == 0 and money >= price:
        #if cert < 1.3 and stocks == 0 and money >= price:    
            print('buyat: money {}, condition {}, price {} , date {}, count {}\n\n'.format(money, cert, price, date, count))
            money, stocks, buy_price = buy(money, price, stocks, buy_price)
            buys = buys + 1
        elif cert >= 8 and stocks == 1:
#             if (price >= buy_price):
#                 print('certain sellat: money {}, condition {}, price {} , count {}\n\n'.format(money, cert, price, date))
#                 money, stocks, sells = sell(money, price, stocks, sells, buy_price)
#                 sells = sells + 1
            holds = holds + 1
        elif cert <=3 and stocks == 1:
            if (price > buy_price):
                print('certain sellat: money {}, condition {}, price {} , count {}\n\n'.format(money, cert, price, date))
                money, stocks, sells = sell(money, price, stocks, sells, buy_price)
                sells = sells + 1
            
        elif cert <= 6 and stocks == 1:
            if (macd < 2):
                print('uncertain sellat: money {}, condition {}, price {} , count {}\n\n'.format(money, cert, price, date))
                money, stocks, sells = sell(money, price, stocks, sells, buy_price)
                uncertsells = uncertsells + 1
        else: 
            holds = holds + 1
        
        if count == (length) and stocks == 1:
            money, stocks, sells = sell(money, price, stocks, sells, buy_price)
            print('--> forced sellat: money {}, condition {}, price {} , count {}\n\n'.format(money, cert, price, date)) 
    
    print(length, money, stocks, sells, uncertain, uncertsells , holds, buys)
    print(count)
    profit = (money - start)/start * 100
    print('profit', profit) 
    count = 0

    return

def certainty_com(df):
    #####++>>>>>>>>>>>>>> make a buy certainty and a sell certianty
    # th
    macda = np.ndarray.flatten(np.array(df.MACD))
    #chakina = np.ndarray.flatten(np.array(df.cmoveA))
    #macdtesta = np.ndarray.flatten(np.array(df.MACDT))
    certa = np.ndarray.flatten(np.array(df.certainty))
    closea = np.ndarray.flatten(np.array(df.Close))
    trendinga = np.ndarray.flatten(np.array(df.trending))
    datea = np.ndarray.flatten(np.array(df.Date))
    
    #funds = int(closea[0]) + 50
    funds = int(closea[0]) + 500
    
    money = funds
    stocks = 0
    bought = 0
    sells = 0 
    holds = 0 
    uncertain = 0
    uncertsells = 0
    buys = 0
    count = 0 
    start = funds
    buy_price = funds
    last_cert = 0
    
    print(funds)
    
    length = len(closea)
    for i in range(length):
        #we should take the avg of trend to see if the majority is above 3 if so it is un relaiable
        macd = macda[i]
        cert = certa[i]
        price = closea[i]
        date = datea[i]
        trend = trendinga[i]
        count = count + 1
        dconfidence = cert - last_cert
        last_cert = cert 
        
        #if (macd > 0 and dconfidence > 0) and (money > price and stocks == 0):
        if (macd > 0) and (money > price and stocks == 0):
            print('buyat: money {}, condition {}, price {} , date {}, count {}\n\n'.format(money, cert, price, date, count))
            money, stocks, buy_price = buy(money, price, stocks, buy_price)
            buys = buys + 1
        elif cert > 7 and (dconfidence > -2 and dconfidence < 2):
            holds = holds + 1
        elif cert > 5 and cert <= 7 and dconfidence < 0 and stocks == 1:
            #un certain conditions, check the trends, check more stuff
            if trend <= 3:  # add nural net predictions here 
                print('uncertain sellat: money {}, condition {}, price {} , count {}\n\n'.format(money, cert, price, date))
                money, stocks, sells = sell(money, price, stocks, sells, buy_price)
                sells = sells + 1
            elif trend > 3:
                holds = holds + 1
        elif cert <= 5 and dconfidence < 0 and stocks == 1:
            if (price > buy_price):
                print('certain sellat: money {}, condition {}, price {} , count {}\n\n'.format(money, cert, price, date))
                money, stocks, sells = sell(money, price, stocks, sells, buy_price)
                sells = sells + 1  
#             if (price < buy_price):
#                 if macd < 0:  # add nural net predictions here 
#                     print('uncertain sellat: money {}, condition {}, price {} , count {}\n\n'.format(money, cert, price, date))
#                     money, stocks, sells = sell(money, price, stocks, sells, buy_price)
#                     sells = sells + 1
#                 elif macd > 0:
#                     holds = holds + 1
#         elif cert < 5 and dconfidence < 0 and stocks == 1:
#             if (price > buy_price):
#                 print('certain sellat: money {}, condition {}, price {} , count {}\n\n'.format(money, cert, price, date))
#                 money, stocks, sells = sell(money, price, stocks, sells, buy_price)
#                 sells = sells + 1
        else: 
            holds = holds + 1
            
        if count == (length) and stocks == 1:
            money, stocks, sells = sell(money, price, stocks, sells, buy_price)
            print('--> forced sellat: money {}, condition {}, price {} , count {}\n\n'.format(money, cert, price, date)) 
        elif stocks == 0 and price > money:
            print('good time to stock hop')
        
        
          
        
    print(length, money, stocks, sells, uncertain, uncertsells , holds, buys)
    print(count)
    profit = (money - start)/start * 100
    print('profit', profit) 
    count = 0

    return

def certainty_com2(df):
    #####++>>>>>>>>>>>>>> make a buy certainty and a sell certianty
    # th
    macda = np.ndarray.flatten(np.array(df.MACD))
    #chakina = np.ndarray.flatten(np.array(df.cmoveA))
    #macdtesta = np.ndarray.flatten(np.array(df.MACDT))
    certa = np.ndarray.flatten(np.array(df.certainty))
    closea = np.ndarray.flatten(np.array(df.Close))
    trendinga = np.ndarray.flatten(np.array(df.trending))
    datea = np.ndarray.flatten(np.array(df.Date))
    
    #funds = int(closea[0]) + 50
    funds = int(closea[0]) + 500
    
    money = funds
    stocks = 0
    bought = 0
    sells = 0 
    holds = 0 
    uncertain = 0
    uncertsells = 0
    buys = 0
    count = 0 
    start = funds
    buy_price = funds
    last_cert = 0
    
    print(funds)
    
    length = len(closea)
    for i in range(length):
        #we should take the avg of trend to see if the majority is above 3 if so it is un relaiable
        macd = macda[i]
        cert = certa[i]
        price = closea[i]
        date = datea[i]
        trend = trendinga[i]
        count = count + 1
        dconfidence = cert - last_cert
        last_cert = cert 
        
        #if (macd > 0 and dconfidence > 0) and (money > price and stocks == 0):
        if (macd > 0) and (money > price and stocks == 0):
            print('buyat: money {}, condition {}, price {} , date {}, count {}\n\n'.format(money, cert, price, date, count))
            money, stocks, buy_price = buy(money, price, stocks, buy_price)
            buys = buys + 1
        elif cert > 8 and (dconfidence > -2 and dconfidence < 2):
            holds = holds + 1
        elif cert > 5 and cert <= 8 and dconfidence < 0 and stocks == 1:
            #un certain conditions, check the trends, check more stuff
            if trend <= 3:  # add nural net predictions here 
                print('uncertain sellat: money {}, condition {}, price {} , count {}\n\n'.format(money, cert, price, date))
                money, stocks, sells = sell(money, price, stocks, sells, buy_price)
                sells = sells + 1
            elif trend > 3:
                holds = holds + 1
        elif cert <= 5 and dconfidence < 0 and stocks == 1:
            if (price > buy_price):
                print('certain sellat: money {}, condition {}, price {} , count {}\n\n'.format(money, cert, price, date))
                money, stocks, sells = sell(money, price, stocks, sells, buy_price)
                sells = sells + 1  
#             if (price < buy_price):
#                 if macd < 0:  # add nural net predictions here 
#                     print('uncertain sellat: money {}, condition {}, price {} , count {}\n\n'.format(money, cert, price, date))
#                     money, stocks, sells = sell(money, price, stocks, sells, buy_price)
#                     sells = sells + 1
#                 elif macd > 0:
#                     holds = holds + 1
#         elif cert < 5 and dconfidence < 0 and stocks == 1:
#             if (price > buy_price):
#                 print('certain sellat: money {}, condition {}, price {} , count {}\n\n'.format(money, cert, price, date))
#                 money, stocks, sells = sell(money, price, stocks, sells, buy_price)
#                 sells = sells + 1
        else: 
            holds = holds + 1
            
        if count == (length) and stocks == 1:
            money, stocks, sells = sell(money, price, stocks, sells, buy_price)
            print('--> forced sellat: money {}, condition {}, price {} , count {}\n\n'.format(money, cert, price, date)) 
        elif stocks == 0 and price > money:
            print('good time to stock hop')
        
        
          
        
    print(length, money, stocks, sells, uncertain, uncertsells , holds, buys)
    print(count)
    profit = (money - start)/start * 100
    print('profit', profit,':::', money - start,'dollars') 
    count = 0

    return

In [0]:
def getData2(ticker, tsfunc = 1, tlen = '1', FoC = 'full'):
    #FoC = full or compact
    apikey = '9OWLY1IEBT5CH6I8'
    intraday = '&interval='+tlen+'min&'
    if tsfunc == 1:
        tssfunc = 'TIME_SERIES_DAILY'
        url = 'https://www.alphavantage.co/query?function=' + tssfunc + '&symbol='+ ticker +'&outputsize='+FoC+'&apikey=' + apikey +'&datatype=csv'
    elif tsfunc == 2:
        tssfunc = 'TIME_SERIES_INTRADAY'
        url = 'https://www.alphavantage.co/query?function=' + tssfunc + '&symbol='+ ticker + intraday + '&outputsize='+FoC+'&apikey=' + apikey +'&datatype=csv'

    csv = requests.get(url).content
    df = pd.read_csv(io.StringIO(csv.decode('utf-8')))
    df.rename(columns = {'timestamp':'Date', 'open':'Open', 'low':'Low', 'high':'High', 'close':'Close', 'volume':'Volume'}, inplace = True)
    df.set_index('Date', inplace = True)
    df = df.sort_index(ascending = True, axis = 0)
    return df


def getData(ticker, day = 20000):
  #we may have to change thit to get the data from more stable source. this will break if the ppl at yahoo change their site evena little
  try:
    print(ticker)
    #start = dt.datetime(1950, 1, 1)
    #start = dt.datetime(2000, 1, 1)
    #start = dt.datetime(2018, 1, 1)
    #start = dt.datetime(2018, 1, 1)
    start = dt.datetime.today() - dt.timedelta(days = day)
    end = dt.datetime.today() #- dt.timedelta(days = 1)
    #end = dt.datetime(2013, 10, 12)
    df = pdr.get_data_yahoo(ticker, start, end)

    while emptychek(df):
        df = pdr.get_data_yahoo(ticker, start, end)

    #print(df.head())
    #print(df.tail())
    return(df)
  except:
    pass

def emptychek(df):
    empty = True
    if empty:
        try:
            print(df.head())
            empty = False
        except:
            empty = True

    print(empty)
    return

def dataSpecify(df):
    #print(df.head(), df.tail())

    df['m50MAvg'] = df['Close'].rolling(window = 50, min_periods = 0).mean()
    df['m20MAvg'] = df['Close'].rolling(window = 20, min_periods = 0).mean()
    df['m10MAvg'] = df['Close'].rolling(window = 10, min_periods = 0).mean()
    df['VolumeMAvg'] = df['Volume'].rolling(window = 50, min_periods = 0).mean()

    df['e50MAvg'] = df['Close'].ewm(span = 50, adjust = False, min_periods = 0).mean()
    df['e26MAvg'] = df['Close'].ewm(span = 26, adjust = False, min_periods = 0).mean()
    df['e12MAvg'] = df['Close'].ewm(span = 12, adjust = False, min_periods = 0).mean()
    #df['e10MAvg'] = df['Close'].ewm(span = 10, adjust = False, min_periods = 0).mean()
    #df['e3MAvg'] = df['Close'].ewm(span = 3, adjust = False, min_periods = 0).mean()
    df['pslope'] = df['e50MAvg'].diff()
    df['MACD'] = df['e12MAvg'] - df['e26MAvg']

    df['MFlowMult'] = ((df.Close - df.Low) - (df.High - df.Close))/(df.High - df.Low)
    df['MFlowVol'] = (df['Volume'])*df['MFlowMult']
    #df['temp'] = df['MFlowVol'].shift(1)
    #df.fillna(0, inplace = True)
    df['ADL'] = df['MFlowVol'].values.cumsum()
    df['ADL'] = df['ADL'].shift(1)
    df['ADX'] = df['ADX']
    df['OBV'] = df['OBV']
    
    df.fillna(0, inplace = True)
    df['e50AAvg'] = df['ADL'].ewm(span = 50, adjust = False, min_periods = 0).mean()
    df['e10MAvg'] = df['ADL'].ewm(span = 10, adjust = False, min_periods = 0).mean()
    df['e3MAvg'] = df['ADL'].ewm(span = 3, adjust = False, min_periods = 0).mean()
    df['ADL_slope'] = df['e50AAvg'].diff()

    df['ADma_slope'] = df['ADL_slope'].ewm(span = 20, adjust = False, min_periods = 0).mean()
    df['Chakin'] = df['e3MAvg'] - df['e10MAvg']
    df['cmoveA'] = df['Chakin'].ewm(span = 50, adjust = False, min_periods = 0).mean()
    df['cmoveA4'] = df['Chakin'].ewm(span = 100, adjust = False, min_periods = 0).mean()
    df['cmoveA3'] = df['Chakin'].ewm(span = 10, adjust = False, min_periods = 0).mean()
    df['cmoveA2'] = df['Chakin'].ewm(span = 26, adjust = False, min_periods = 0).mean() #increasing means uptend
    #ch moving avg positiv and (chakin positive, trending up still, negative gonna start trending down soon)
                                                      #ch moving avg p and chakin n = sell
    #ch moving avg negtive and (chakin positive, trending up , negative trending down still)
    #ch moving avg n and chakin p = buy
    # chakin slope is similar to chakin itslef

    df['MACDT'] = df['cmoveA3'] - df['cmoveA4'] # just a check means nothing for now
    #df['chakinslope'] = df['cmoveA2'].diff()
    #df.drop(['temp'], inplace = True)

#     df['cmoveA_certinty'] = df['cmoveA2'].apply(cmoveA_cert)
#     df['chakin_certinty'] = df['Chakin'].apply(cmoveA_cert)
#     df['MACD_certinty'] = df['MACD'].apply(MACD_cert)
#     df['MACDT_certinty'] = df['MACDT'].apply(MACDT_cert)
#     df['vol_certinty'] = df['Volume'] > df['VolumeMAvg']
#     df['vol_certinty'] = df['vol_certinty'].apply(vol_cert)
    
#     df['MACDT2_certinty'] = df['MACDT'].apply(MACDT2_cert)
    
#     df['certainty'] = df['chakin_certinty']+df['MACD_certinty']+df['MACDT_certinty']+df['vol_certinty']
#     df['trending'] = df['cmoveA_certinty']*2

    print(df.head(), df.tail())
    #df['ADL_slope'].plot()
    df['pslope'].plot()
    plt.show()
    return df

def Rsi(df, alpha = True):
    window_len = 14
    df['delClose'] = df['Close'].diff()
    df['delUp'] = df['delClose'].apply(Up)
    df['delDown'] = df['delClose'].apply(Down)
    if alpha: 
        # fix the moving avgs
        df['avgUp'] = df['delUp'].ewm(alpha = 1/window_len, adjust = False).mean()
        df['avgDown'] = df['delDown'].ewm(alpha = 1/window_len, adjust = False).mean()
    else:
        df['avgUp'] = df['delUp'].ewm(span = window_len, adjust = False).mean()
        df['avgDown'] = df['delDown'].ewm( span = window_len, adjust = False).mean()
    
    df['RS'] = df['avgUp']/df['avgDown']
    df['RSI'] = 100 - 100/(1 + df['RS'])
    
    print(df[['Close', 'delClose', 'delUp','delDown','avgUp', 'avgDown', 'RS', 'RSI']].head(20))
    df['RSI'].plot()
    plt.grid(which='both', linestyle='-', linewidth='0.2', color='red')
    plt.show()
    return df

def Up(value):
    return (value if value > 0 else 0)
        
def Down(value):
    return (abs(value) if value < 0 else 0)
    
def scale (df):
#     for col in df.columns:
#         if col == 'MACD' or col != 'Chakin' or col != 'cmoveA2' or col == 'MACDT':
#             scaler = preprocessing.StandardScaler()
#             df[f"{col}s"] = scaler.fit_transform(np.array(df[col].values.reshape(-1, 1)))
    for col in df.columns:
        scaler = preprocessing.StandardScaler()
        df[f"{col}s"] = scaler.fit_transform(np.array(df[col].values.reshape(-1, 1)))
            
#     scaler = preprocessing.StandardScaler()
#     df['MACDs'] = scaler.fit_transform(np.array(df['MACD'].values.reshape(-1, 1)))
    
#     scaler = preprocessing.StandardScaler()
#     df['Chakins'] = scaler.fit_transform(np.array(df['Chakin'].values.reshape(-1, 1)))
    
#     scaler = preprocessing.StandardScaler()
#     df['cmoveA2s'] = scaler.fit_transform(np.array(df['cmoveA2'].values.reshape(-1, 1)))

#     scaler = preprocessing.StandardScaler()
#     df['MACDTs'] = scaler.fit_transform(np.array(df['MACDT'].values.reshape(-1, 1)))
    return df

def certainty(df):
    df['cmoveA_certinty'] = df['cmoveA2s'].apply(cmoveA_cert)
    df['cmoveA_trending'] = df['cmoveA2s'].apply(cmoveA_trending)
    df['chakin_certinty'] = df['Chakins'].apply(cmoveA_cert)
    df['MACD_certinty'] = df['MACDs'].apply(MACD_cert)
    df['MACDT_certinty'] = df['MACDTs'].apply(MACDT_cert)
    df['vol_certinty'] = df['Volumes'] > df['VolumeMAvg']
    df['vol_certinty'] = df['vol_certinty'].apply(vol_cert)
    df['RSI_certainty'] = df['RSI'].apply(rsi_cert)
    df['ADma_sl_certainty'] = df['ADma_slope'].apply(cmoveA_trending) 
    df['pslopes_cert'] = df['pslopes'].apply(cmoveA_trending)*3 / 5
    
    df['MACDT2_certinty'] = df['MACDTs'].apply(MACDT2_cert)
    
    df['certainty'] = df['chakin_certinty'] + df['MACD_certinty'] + df['RSI_certainty'] + df['pslopes_cert']
    #df['certainty'] = df['chakin_certinty'] + df['MACD_certinty'] + df['MACDT_certinty'] + df['vol_certinty'] + df['RSI_certainty']
    df['trending'] = (df['cmoveA_trending'] + df['ADma_sl_certainty']) * df['pslopes'] #df['ADma_slope']*5
    #df['trending'] = df['cmoveA_trending'] * df['ADma_slope']  #df['ADma_sl_certainty']
    
    df['trendingma'] = df['trending'].ewm(span = 26, adjust = False, min_periods = 0).mean()
    df['trendingslope'] = df['trendingma'].diff().ewm(span = 10, adjust = False, min_periods = 0).mean()
    df['trending'].plot()
    
    df['trendingma'].plot()
    (df['trendingslope']*20).plot()
    plt.grid(which='both', linestyle='-', linewidth='0.2', color='red')
    plt.show()
    
    df['trendingma'].diff().plot()
    (df['trendingslope']).plot()
    plt.grid(which='both', linestyle='-', linewidth='0.2', color='red')
    plt.show()
    
    return df
    
def getGraphSpecified2(df):
    df.reset_index(inplace = True)
    #print(df.head())
    fig1 = plt.figure()
    ax1 = plt.subplot2grid((12,4),(0,0), rowspan = 4, colspan = 4)
    ax1.plot(df.Date, df.e26MAvg)
    ax1.plot(df.Date, df.e12MAvg)
    ax1.plot(df.Date, df.Close)

    ax1.grid(which='both', linestyle='-', linewidth='0.2', color='red')
    ax1.grid(which='minor', linestyle=':', linewidth='0.2', color='black')
    plt.ylabel('Stock Price')

    

#     ax3 = plt.subplot2grid((12,4), (6,0), sharex = ax1, rowspan = 2, colspan = 4)
#     #ax3.plot(df.Date, df.Chakin)
#     #ax3.plot(df.Date, df.cmoveA)
#     #ax3.plot(df.Date, df.cmoveA2)
#     ax3.plot(df.Date, df.cmoveA3)
#     ax3.plot(df.Date, df.cmoveA4)
#     ax3.grid(which='both', linestyle='-', linewidth='0.2', color='red')
#     ax3.grid(which='minor', linestyle=':', linewidth='0.2', color='black')
#     plt.ylabel('chakin')
    
#     ax6 = plt.subplot2grid((12,4), (4,0), sharex = ax1, rowspan = 4, colspan = 4)
#     ax6.plot(df.Date, df.certainty)
#     ax6.grid(which='both', linestyle='-', linewidth='0.2', color='red')
#     ax6.grid(which='minor', linestyle='-', linewidth='0.2', color='black')
    
    ax7 = plt.subplot2grid((12,4), (4,0), sharex = ax1, rowspan = 8, colspan = 4)
    ax7.plot(df.Date, df.trending)
    ax7.plot(df.Date, df.trendingma)
    ax7.plot(df.Date, df.trendingslope*20)
    
    ax7.grid(which='both', linestyle='-', linewidth='0.2', color='red')
    ax7.grid(which='minor', linestyle='-', linewidth='0.2', color='black')
    
    plt.show()
    return

def getGraphSpecified(df):
    df.reset_index(inplace = True)
    #print(df.head())
    fig1 = plt.figure()
    ax1 = plt.subplot2grid((12,4),(0,0), rowspan = 4, colspan = 4)
    ax1.plot(df.Date, df.e26MAvg)
    ax1.plot(df.Date, df.e12MAvg)
    ax1.plot(df.Date, df.Close)

    ax1.grid(which='both', linestyle='-', linewidth='0.2', color='red')
    ax1.grid(which='minor', linestyle=':', linewidth='0.2', color='black')
    plt.ylabel('Stock Price')

    ax2 = plt.subplot2grid((12,4), (4,0), sharex = ax1, rowspan = 2, colspan = 4)
    ax2.bar(df.Date, df.Volume)
    ax2.plot(df.Date, df.VolumeMAvg)
    ax2.plot(df.Date, df.ADL)
    ax2.plot(df.Date, df.e10MAvg)
    ax2.grid(which='both', linestyle='-', linewidth='0.2', color='red')
    ax2.grid(which='minor', linestyle=':', linewidth='0.2', color='black')
    plt.ylabel('Volume + ADL')

    ax3 = plt.subplot2grid((12,4), (6,0), sharex = ax1, rowspan = 2, colspan = 4)
    #ax3.plot(df.Date, df.Chakin)
    #ax3.plot(df.Date, df.cmoveA)
    #ax3.plot(df.Date, df.cmoveA2)
    ax3.plot(df.Date, df.cmoveA3)
    ax3.plot(df.Date, df.cmoveA4)
    ax3.grid(which='both', linestyle='-', linewidth='0.2', color='red')
    ax3.grid(which='minor', linestyle=':', linewidth='0.2', color='black')
    plt.ylabel('chakin')

    ax4 = plt.subplot2grid((12,4), (8,0), sharex = ax1, rowspan = 1, colspan = 4)
    ax4.bar(df.Date, df.MACD)
    ax4.grid(which='both', linestyle='-', linewidth='0.2', color='red')
    ax4.grid(which='minor', linestyle=':', linewidth='0.2', color='black')
    plt.ylabel('MACD')

    ax5 = plt.subplot2grid((12,4), (9,0), sharex = ax1, rowspan = 1, colspan = 4)
    ax5.bar(df.Date, df.MACDT)
    #ax5.plot(df.Date, df.chakinslope)
    #ax5.grid(True)
    ax5.grid(which='both', linestyle='-', linewidth='0.2', color='red')
    ax5.grid(which='minor', linestyle='-', linewidth='0.2', color='black')
    plt.ylabel('MACDT')
    
    ax6 = plt.subplot2grid((12,4), (10,0), sharex = ax1, rowspan = 1, colspan = 4)
    ax6.plot(df.Date, df.certainty)
    ax6.grid(which='both', linestyle='-', linewidth='0.2', color='red')
    ax6.grid(which='minor', linestyle='-', linewidth='0.2', color='black')
    
    ax7 = plt.subplot2grid((12,4), (11,0), sharex = ax1, rowspan = 1, colspan = 4)
    ax7.plot(df.Date, df.trending)
    ax7.grid(which='both', linestyle='-', linewidth='0.2', color='red')
    ax7.grid(which='minor', linestyle='-', linewidth='0.2', color='black')
    
    plt.show()
    return

#def slope():


def buy(money, price, stocks, buys):
    money = money - price 
    stocks = stocks + 1
    buys = price
    return money, stocks, buys

def sell(money, price, stocks, sells, buy_price):
    tax = (price - buy_price) * 0.25
    money = money + price - tax
    stocks = stocks - 1
    return money, stocks, sells

def hold(holds):
    pass

def cmoveA_trending(num):
#     if num > 0:
#         num = 3
#     else:
#         num = 0
        
    num = 5/(1 + np.exp(-num))
    #num = 1/(1 + np.exp(-num))
    #num = 5 * np.tanh(num)
    return num

def cmoveA_cert(num):
#     if num > 0:
#         num = 3
#     else:
#         num = 0
        
    #num = 3/(1 + exp(-num/1000000))
    num = 2/(1 + exp(-num))
    return num


def MACD_cert(num):
#     if num > 0:
#         num = 4
#     else:
#         num = 0
    num = 3/(1 + exp(-num))
    return num

def MACDT_cert(num):
#     if num > 0:
#         num = 1
#     else:
#         num = 0
    num = 1/(1 + exp(-num))
    return num

def MACDT2_cert(num):
#     if num > 0:
#         num = 3
#     else:
#         num = 0
    num = 3/(1 + exp(-num))
    return num

# def dolar_stab():
#     #stability of the dollar
#     pass

def rsi_cert(num): 
    if num > 70:
        num = 1
    elif num < 30:
        num = -1
    else:
        num = 0
    return num

def vol_cert(vol):
    if True:
        num = 1
    else:
        num = 0
    return num

def cert(df):
    print('\n')
    print((df.certainty).values)
    print('\n')
    
    
def trend(df):
    print('\n')
    print((df.trending).values)
    print('\n')
    
    
    

In [0]:
'''
philosiphy: 
these should be with held regardless of price 
if we loose more than x dollars we sell and 


buy:
if certainty changes from low to high, at that instant we buy, and/or macd is positive
    dcertainty > 5 and certainty > 5 buy i think i am not sure

sell and hold
if certainty is high we hold, i would say like 70 percent certainty, like 7/9 or 7/10
if Certainty is low like 3/9 or 3/10 or 4/9 or 4/10 btween 30 and 40 percent we should sell with certainty that nothing good will follow
    
if certainty is between 30 and 70, this is the uncertain range. 
    if the trend is uptrend: we hold
    if else we sell to minimize loss
    
in any other situation we hold 


'''

#def trail(df):
    




'\nphilosiphy: \nthese should be with held regardless of price \nif we loose more than x dollars we sell and \n\n\nbuy:\nif certainty changes from low to high, at that instant we buy, and/or macd is positive\n    dcertainty > 5 and certainty > 5 buy i think i am not sure\n\nsell and hold\nif certainty is high we hold, i would say like 70 percent certainty, like 7/9 or 7/10\nif Certainty is low like 3/9 or 3/10 or 4/9 or 4/10 btween 30 and 40 percent we should sell with certainty that nothing good will follow\n    \nif certainty is between 30 and 70, this is the uncertain range. \n    if the trend is uptrend: we hold\n    if else we sell to minimize loss\n    \nin any other situation we hold \n\n\n'

In [0]:
def certainty_com_chakin(df):
    #####++>>>>>>>>>>>>>> make a buy certainty and a sell certianty
    # th
    macda = np.ndarray.flatten(np.array(df.MACD))
    chakina = np.ndarray.flatten(np.array(df.Chakin))
    #macdtesta = np.ndarray.flatten(np.array(df.MACDT))
    certa = np.ndarray.flatten(np.array(df.certainty))
    closea = np.ndarray.flatten(np.array(df.Close))
    trendinga = np.ndarray.flatten(np.array(df.trending))
    datea = np.ndarray.flatten(np.array(df.Date))
    
    #funds = int(closea[0]) + 50
    funds = int(closea[0]) + 500
    
    money = funds
    stocks = 0
    bought = 0
    sells = 0 
    holds = 0 
    uncertain = 0
    uncertsells = 0
    buys = 0
    count = 0 
    start = funds
    buy_price = funds
    last_cert = 0
    
    print(funds)
    
    length = len(closea)
    for i in range(length):
        #we should take the avg of trend to see if the majority is above 3 if so it is un relaiable
        macd = macda[i]
        cert = certa[i]
        price = closea[i]
        chakin = chakina[i]
        date = datea[i]
        trend = trendinga[i]
        count = count + 1
        dconfidence = cert - last_cert
        last_cert = cert 
        
        if (chakin < 0 and cert < 5) and (money > price and stocks == 0):
            print('buyat: money {}, condition {}, price {} , date {}, count {}\n\n'.format(money, cert, price, date, count))
            money, stocks, buy_price = buy(money, price, stocks, buy_price)
            buys = buys + 1
        elif cert > 6 and (dconfidence > -3 and dconfidence < 3):
            holds = holds + 1
        elif cert > 3 and cert <= 6 and stocks == 1:
            #un certain conditions, check the trends, check more stuff
            if (price > buy_price): # add nural net predictions here 
                print('certain sellat: money {}, condition {}, price {} , count {}\n\n'.format(money, cert, price, date))
                money, stocks, sells = sell(money, price, stocks, sells, buy_price)
                sells = sells + 1
            elif chakin > 0:
                holds = holds + 1
        elif cert <= 3 and dconfidence < 0 and stocks == 1:
            if (price > buy_price):
                print('certain sellat: money {}, condition {}, price {} , count {}\n\n'.format(money, cert, price, date))
                money, stocks, sells = sell(money, price, stocks, sells, buy_price)
                sells = sells + 1  
            #loss minimization
            elif (chakin > 0):
                #un certain conditions, check the trends, check more stuff
                  # add nural net predictions here 
                print('uncertain sellat: money {}, condition {}, price {} , count {}\n\n'.format(money, cert, price, date))
                money, stocks, sells = sell(money, price, stocks, sells, buy_price)
                sells = sells + 1
#             elif chakin < 0:
#                 holds = holds + 1 
            else:
                holds = holds + 1
#         elif cert < 5 and dconfidence < 0 and stocks == 1:
#             if (price > buy_price):
#                 print('certain sellat: money {}, condition {}, price {} , count {}\n\n'.format(money, cert, price, date))
#                 money, stocks, sells = sell(money, price, stocks, sells, buy_price)
#                 sells = sells + 1
        else: 
            holds = holds + 1
            
        if count == (length) and stocks == 1:
            money, stocks, sells = sell(money, price, stocks, sells, buy_price)
            print('--> forced sellat: money {}, condition {}, price {} , count {}\n\n'.format(money, cert, price, date)) 
        elif stocks == 0 and price > money:
            print('good time to stock hop')
            
    print(length, money, stocks, sells, uncertain, uncertsells , holds, buys)
    print(count)
    profit = (money - start)/start * 100
    print('profit', profit,':::', money - start,'dollars')  
    count = 0

    return
        

In [0]:
def trending_strat3(df):
    closea = np.ndarray.flatten(np.array(df.Close))
    #trendinga = np.ndarray.flatten(np.array(df.trendingma))
    trendinga = np.ndarray.flatten(np.array(df.trending))
    trendingslopea = np.ndarray.flatten(np.array(df.trendingslope))
    datea = np.ndarray.flatten(np.array(df.Date))
    macda = np.ndarray.flatten(np.array(df.MACD))
    certa = np.ndarray.flatten(np.array(df.certainty))
    chakina = np.ndarray.flatten(np.array(df.Chakin))
    
    #funds = int(closea[0]) + 50
    funds = int(closea[0]) + 500
    
    money = funds
    stocks = 0
    bought = 0
    sells = 0 
    holds = 0 
    uncertain = 0
    uncertsells = 0
    buys = 0
    count = 0 
    start = funds
    buy_price = funds
    last_cert = 0
    
    print(funds)
    
    length = len(closea)
    for i in range(length):
        trend = trendinga[i]
        deltrend = trendingslopea[i]
        price = closea[i]
        macd = macda[i]
        date = datea[i]
        cert = certa[i]
        chakin = chakina[i]
        count = count + 1
        
        
        if stocks == 0:
            #if macd > 0 and money > price:
            #if trend > 0 and macd > 0 and deltrend > 0 and money > price:
            if trend > 0 and chakin > 0 and deltrend > 0 and money > price:
                print('buyat: money {}, trend {} {}, price {} , date {}, count {}\n\n'.format(money, trend, deltrend, price, date, count))
                money, stocks, buy_price = buy(money, price, stocks, buy_price)
                buys = buys + 1
        elif stocks == 1:
            if trend > 0 and deltrend > 0 and price > buy_price: 
                holds = holds + 1
            elif trend > 0 and deltrend > 0 and price < buy_price: 
                print('uncertain sellat: money {}, condition {} {}, price {} , count {}\n\n'.format(money, trend, deltrend, price, date))
                money, stocks, sells = sell(money, price, stocks, sells, buy_price)
                sells = sells + 1
            elif trend > 0 and deltrend < 0:
                print('kinda certain sellat: money {}, condition {} {}, price {} , count {}\n\n'.format(money, trend, deltrend, price, date))
                money, stocks, sells = sell(money, price, stocks, sells, buy_price)
                sells = sells + 1
            elif trend < 0:
                print('certain sellat: money {}, condition {} {}, price {} , count {}\n\n'.format(money, trend, deltrend, price, date))
                money, stocks, sells = sell(money, price, stocks, sells, buy_price)
                sells = sells + 1
        else:
            holds = holds + 1
                
        
        if count == (length) and stocks == 1:
            money, stocks, sells = sell(money, price, stocks, sells, buy_price)
            print('--> forced sellat: money {}, condition {}, price {} , count {}\n\n'.format(money, cert, price, date)) 
        elif stocks == 0 and price > money:
            print('good time to stock hop')

    print(length, money, stocks, sells, uncertain, uncertsells , holds, buys)
    print(count)
    profit = (money - start)/start * 100
    print('profit', profit,':::', money - start,'dollars')
    count = 0
    return


In [0]:
#     df['trendingma'] = df['trending'].ewm(span = 26, adjust = False, min_periods = 0).mean()
#     df['trendingslope'] = df['trendingma'].diff().ewm(span = 10, adjust = False, min_periods = 0).mean()
#     df['trending'].plot()
    
#     df['trendingma'].plot()
#     (df['trendingslope']*20).plot()
#     plt.grid(which='both', linestyle='-', linewidth='0.2', color='red')
#     plt.show()
    
#     df['trendingma'].diff().plot()
#     (df['trendingslope']).plot()
#     plt.grid(which='both', linestyle='-', linewidth='0.2', color='red')
#     plt.show()
def trending_strat(df):
    closea = np.ndarray.flatten(np.array(df.Close))
    trendinga = np.ndarray.flatten(np.array(df.trendingma))
    #trendinga = np.ndarray.flatten(np.array(df.trending))
    trendingslopea = np.ndarray.flatten(np.array(df.trendingslope))
    datea = np.ndarray.flatten(np.array(df.Date))
    #funds = int(closea[0]) + 50
    funds = int(closea[0]) + 500
    
    
    money = funds
    stocks = 0
    bought = 0
    sells = 0 
    holds = 0 
    uncertain = 0
    uncertsells = 0
    buys = 0
    count = 0 
    start = funds
    buy_price = funds
    last_cert = 0
    
    print(funds)
    
    length = len(closea)
    for i in range(length):
        trend = trendinga[i]
        deltrend = trendingslopea[i]
        price = closea[i]
        date = datea[i]
        count = count + 1
        
        
        if stocks == 0:
            if trend > 0 and deltrend > 0 and money > price:
                print('buyat: money {}, trend {} {}, price {} , date {}, count {}\n\n'.format(money, trend, deltrend, price, date, count))
                money, stocks, buy_price = buy(money, price, stocks, buy_price)
                buys = buys + 1
                
        elif stocks == 1:
            if trend > 0 and deltrend > 0: 
                holds = holds + 1
            elif trend > 0 and deltrend < 0:
                print('kinda certain sellat: money {}, condition {} {}, price {} , count {}\n\n'.format(money, trend, deltrend, price, date))
                money, stocks, sells = sell(money, price, stocks, sells, buy_price)
                sells = sells + 1
            elif trend < 0:
                print('certain sellat: money {}, condition {} {}, price {} , count {}\n\n'.format(money, trend, deltrend, price, date))
                money, stocks, sells = sell(money, price, stocks, sells, buy_price)
                sells = sells + 1
        else:
            holds = holds + 1
                
        
        if count == (length) and stocks == 1:
            money, stocks, sells = sell(money, price, stocks, sells, buy_price)
            print('--> forced sellat: money {}, condition {}, price {} , count {}\n\n'.format(money, cert, price, date)) 
        elif stocks == 0 and price > money:
            print('good time to stock hop')

    print(length, money, stocks, sells, uncertain, uncertsells , holds, buys)
    print(count)
    profit = (money - start)/start * 100
    print('profit', profit,':::', money - start,'dollars') 
    count = 0
    return


In [0]:
def trending_strat2(df):
    closea = np.ndarray.flatten(np.array(df.Close))
    trendinga = np.ndarray.flatten(np.array(df.trendingma))
    #trendinga = np.ndarray.flatten(np.array(df.trending))
    trendingslopea = np.ndarray.flatten(np.array(df.trendingslope))
    datea = np.ndarray.flatten(np.array(df.Date))
    macda = np.ndarray.flatten(np.array(df.MACD))
    
    #funds = int(closea[0]) + 50
    funds = int(closea[0]) + 500
    
    money = funds
    stocks = 0
    bought = 0
    sells = 0 
    holds = 0 
    uncertain = 0
    uncertsells = 0
    buys = 0
    count = 0 
    start = funds
    buy_price = funds
    p_yesterday = 0 
    sell_price = 0
    last_cert = 0
    
    print(funds)
    
    length = len(closea)
    for i in range(length):
        trend = trendinga[i]
        deltrend = trendingslopea[i]
        price = closea[i]
        macd = macda[i]
        date = datea[i]
        count = count + 1
        
        
        if stocks == 0 and price > p_yesterday:
            #if macd > 0 and money > price:
            sell_price = 0
            if macd > 0 and money > price:# and deltrend > 0:
                print('buyat: money {}, trend {} {}, price {} , date {}, count {}\n\n'.format(money, trend, deltrend, price, date, count))
                money, stocks, buy_price = buy(money, price, stocks, buy_price)
                buys = buys + 1
        elif stocks == 1:
            if trend > 0 and deltrend > 0 and price > buy_price: 
                holds = holds + 1
                uncertain = 0
            elif trend > 0 and deltrend > 0 and price < buy_price: 
                uncertain = uncertain + 1
            elif trend > 0 and deltrend < 0:
                print('kinda certain sellat: money {}, condition {} {}, price {} , count {}\n\n'.format(money, trend, deltrend, price, date))
                money, stocks, sells = sell(money, price, stocks, sells, buy_price)
                sells = sells + 1
                uncertain = 0
            elif trend < 0:
                print('certain sellat: money {}, condition {} {}, price {} , count {}\n\n'.format(money, trend, deltrend, price, date))
                money, stocks, sells = sell(money, price, stocks, sells, buy_price)
                sells = sells + 1
                uncertain = 0
        else:
            holds = holds + 1
            
        if uncertain == 2:
            print('uncertain sellat: money {}, condition {} {}, price {} , count {}\n\n'.format(money, trend, deltrend, price, date))
            sell_price = price
            money, stocks, sells = sell(money, price, stocks, sells, buy_price)
            sells = sells + 1
            uncertain = 0
        elif count == (length) and stocks == 1:
            money, stocks, sells = sell(money, price, stocks, sells, buy_price)
            print('--> forced sellat: money {}, condition {}, price {} , count {}\n\n'.format(money, cert, price, date)) 
        elif stocks == 0 and price > money:
            print('good time to stock hop')
        p_yesterday = price

    print(length, money, stocks, sells, uncertain, uncertsells , holds, buys)
    print(count)
    profit = (money - start)/start * 100
    print('profit', profit,':::', money - start,'dollars')
    count = 0
    return


In [0]:
def trending_strat2(df):
    closea = np.ndarray.flatten(np.array(df.Close))
    trendinga = np.ndarray.flatten(np.array(df.trendingma))
    #trendinga = np.ndarray.flatten(np.array(df.trending))
    trendingslopea = np.ndarray.flatten(np.array(df.trendingslope))
    datea = np.ndarray.flatten(np.array(df.Date))
    macda = np.ndarray.flatten(np.array(df.MACD))
    
    #funds = int(closea[0]) + 50
    funds = int(closea[0]) + 500
    
    money = funds
    stocks = 0
    bought = 0
    sells = 0 
    holds = 0 
    uncertain = 0
    uncertsells = 0
    buys = 0
    count = 0 
    start = funds
    buy_price = funds
    p_yesterday = 0 
    sell_price = 0
    last_cert = 0
    
    print(funds)
    
    length = len(closea)
    for i in range(length):
        trend = trendinga[i]
        deltrend = trendingslopea[i]
        price = closea[i]
        macd = macda[i]
        date = datea[i]
        count = count + 1
        
        
        if stocks == 0 and price > p_yesterday:
            #if macd > 0 and money > price:
            sell_price = 0
            if macd > 0 and deltrend > 0 and money > price:# and deltrend > 0:
                print('buyat: money {}, trend {} {}, price {} , date {}, count {}\n\n'.format(money, trend, deltrend, price, date, count))
                money, stocks, buy_price = buy(money, price, stocks, buy_price)
                buys = buys + 1
        elif stocks == 1:
            if deltrend > 0 and price > buy_price: 
                holds = holds + 1
                uncertain = 0
            elif trend > 0 and deltrend > 0 and price < buy_price: 
                uncertain = uncertain + 1
            elif trend > 0 and deltrend < 0:
                print('kinda certain sellat: money {}, condition {} {}, price {} , count {}\n\n'.format(money, trend, deltrend, price, date))
                money, stocks, sells = sell(money, price, stocks, sells, buy_price)
                sells = sells + 1
                uncertain = 0
            elif trend < 0:
                print('certain sellat: money {}, condition {} {}, price {} , count {}\n\n'.format(money, trend, deltrend, price, date))
                money, stocks, sells = sell(money, price, stocks, sells, buy_price)
                sells = sells + 1
                uncertain = 0
        else:
            holds = holds + 1
            
        if uncertain == 2:
            print('uncertain sellat: money {}, condition {} {}, price {} , count {}\n\n'.format(money, trend, deltrend, price, date))
            sell_price = price
            money, stocks, sells = sell(money, price, stocks, sells, buy_price)
            sells = sells + 1
            uncertain = 0
        elif count == (length) and stocks == 1:
            money, stocks, sells = sell(money, price, stocks, sells, buy_price)
            print('--> forced sellat: money {}, condition {}, price {} , count {}\n\n'.format(money, cert, price, date)) 
        elif stocks == 0 and price > money:
            print('good time to stock hop')
        p_yesterday = price

    print(length, money, stocks, sells, uncertain, uncertsells , holds, buys)
    print(count)
    profit = (money - start)/start * 100
    print('profit', profit,':::', money - start,'dollars')
    count = 0
    return


In [0]:
# use a nural net to determine what numbers to use as sell buy and hold
#dr = getData('AAPL', day = 500)
#dr = getData('AMZN', day = 500)
#dr = getData('TSLA', day = 500) # doesnt work
#dr = getData('GOOG', day = 500)
#dr = getData('^GSPC', day = 500) #==> we need to make the scaling dependent on the company
#dr = getData('NFLX', day = 500)
#dr = getData('AMD', day = 500) #==> we need to make the scaling dependent on the company
#dr = getData('ROKU', day = 500)
dr = getData('^VIX', day = 500) #==> we need to make the scaling dependent on the company


^VIX
                 High    Low   Open      Close  Volume  Adj Close
Date                                                             
2018-01-29  13.840000  11.68  11.71  13.840000       0  13.840000
2018-01-30  15.420000  13.88  13.93  14.790000       0  14.790000
2018-01-31  14.440000  13.41  14.23  13.540000       0  13.540000
2018-02-01  14.300000  12.50  13.05  13.470000       0  13.470000
2018-02-02  17.860001  13.64  13.64  17.309999       0  17.309999
False


In [0]:
df = dr
df = dataSpecify(df)
#df = Rsi(df, alpha = False)

df = scale(df)
df = Rsi(df)

KeyError: ignored

In [0]:
df = certainty(df)
#print(df.head(10))
getGraphSpecified2(df)
df.reset_index(inplace = True)
#print(df.certainty.values)
# cert(df)
# trend(df)

In [0]:
# certainty_basic(df)
# print('\n\n\n\n\n')

# certainty_complex(df)
# # on some companies this one speed trades (ex = tesla), it makes a profit but it is super in efficient
# print('\n\n\n\n\n')

# certainty_com(df)
# print('\n\n\n\n\n')

# #certainty_com2(df)


In [0]:
#speed seller more risky because it doesnt sell when uncertain it holds
certainty_com_chakin(df)
#certainty_com_cha(df)
#fix


In [0]:
# MACD_strat(df)
# print('\n\n\n')
# MACD_basic(df)

In [0]:
trending_strat(df)

In [0]:
trending_strat2(df)

In [0]:
pip install alpha_vantage

In [0]:
#from alpha_vantage.techindicators import TechIndicators
#from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt
import csv
import pandas as pd

#ts = TimeSeries(key='IA5WT3NKED90E2DF', output_format='pandas')
#data, metadata = ts.get_obv(symbol='NASDAQ:AAPL', interval='1mmin') #chage interval
#df = ts.get_daily("APPL")

tlen = '2'
print( '&interval='+tlen+'min&')
def OBV(data, interval, close_values='Close', volume_values='Volume'):
    for index, row in data.iterrows():

        if index > 0:
            last_obv = data.at[index - 1, 'obv']
            if row[close_values] > data.at[index - 1, close_values]:
                current_obv = last_obv + row[volume_values]
            elif row[close_values] < data.at[index - 1, close_values]:
                current_obv = last_obv - row[volume_values]
            else:
                current_obv = last_obv
        else:
            last_obv = 0
            current_obv = row[volume_values]

        data.set_value(index, 'obv', current_obv)

    data['obv_ema ' + str(interval)] = data['obv'].ewm(ignore_na=False, min_periods=0, com=interval, adjust=True).mean()
    
    return data





&interval=2min&
                 High        Low       Open  ...     MFlowMult  MFlowVol  ADL
Date                                         ...                             
2018-01-29  13.840000  11.680000  11.710000  ...  1.000000e+00       0.0  NaN
2018-01-30  15.420000  13.880000  13.930000  ...  1.818180e-01       0.0  0.0
2018-01-31  14.440000  13.410000  14.230000  ... -7.475725e-01      -0.0  0.0
2018-02-01  14.300000  12.500000  13.050000  ...  7.777796e-02       0.0  0.0
2018-02-02  17.860001  13.640000  13.640000  ...  7.393360e-01       0.0  0.0
2018-02-05  38.799999  16.799999  18.440001  ...  8.654546e-01       0.0  0.0
2018-02-06  50.299999  22.420000  37.320000  ... -4.576758e-01      -0.0  0.0
2018-02-07  31.639999  21.170000  31.379999  ...  2.531041e-01       0.0  0.0
2018-02-08  36.169998  24.410000  27.290001  ...  5.391157e-01       0.0  0.0
2018-02-09  41.060001  27.730000  32.180000  ... -8.004502e-01      -0.0  0.0
2018-02-12  29.700001  24.420000  27.250000  ...

TypeError: ignored

In [0]:
import logging
import pandas as pd
import numpy as np
def ADX(df, n, n_ADX):
    i = 0
    UpI = []
    DoI = []
    while i + 1 <= df.index[-1]:
        UpMove = df.loc[i + 1, 'High'] - df.loc[i, 'High']
        DoMove = df.loc[i, 'Low'] - df.loc[i + 1, 'Low']
        if UpMove > DoMove and UpMove > 0:
            UpD = UpMove
        else:
            UpD = 0
        UpI.append(UpD)
        if DoMove > UpMove and DoMove > 0:
            DoD = DoMove
        else:
            DoD = 0
        DoI.append(DoD)
        i = i + 1
    i = 0
    TR_l = [0]
    while i < df.index[-1]:
        TR = max(df.loc[i + 1, 'High'], df.loc[i, 'Close']) - min(df.loc[i + 1, 'Low'], df.loc[i, 'Close'])
        TR_l.append(TR)
        i = i + 1
    TR_s = pd.Series(TR_l)
    ATR = pd.Series(TR_s.ewm(span=n, min_periods=n).mean())
    UpI = pd.Series(UpI)
    DoI = pd.Series(DoI)
    PosDI = pd.Series(UpI.ewm(span=n, min_periods=n).mean() / ATR)
    NegDI = pd.Series(DoI.ewm(span=n, min_periods=n).mean() / ATR)
    ADX = pd.Series((abs(PosDI - NegDI) / (PosDI + NegDI)).ewm(span=n_ADX, min_periods=n_ADX).mean(),
                    name='ADX_' + str(n) + '_' + str(n_ADX))
    df = df.join(ADX)
    return df


